In [1]:
# import_sqlite_to_postgres.py (HOÀN HẢO - ESCAPE %s)
import sqlite3
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import os
from passlib.context import CryptContext
import asyncpg
import warnings


In [2]:
warnings.filterwarnings("ignore", category=UserWarning, module="passlib")

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Tạo mật khẩu mã hóa
plain_password = "111111"
hashed_password = pwd_context.hash(plain_password)
print(hashed_password)

$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5LnaMOOI74iFPfaS


In [3]:
# ===================== CẤU HÌNH =====================
SQLITE_DB = r'\\172.23.10.230\map-eng\8. BAO CAO THIET BI EOL\6. Data quan trong\12. DataWIP\Managementdata\Machine\Inventory_management_EOL.db'
TABLE_INPUT = "input_inventory"
TABLE_OUTPUT = "output_inventory"
TABLE_USER = "user_inventory"

TEAM_ID = 1

POSTGRES_DSN = "host=172.23.8.153 dbname=warehouse_db user=postgres password=wsepc port=5432 sslmode=disable"

# COLUMNS_TO_MELT = [
#     "model", "process", "groups", "material",
#     "unit", "storage_location", "status"
# ]

In [4]:
# lấy tên tất cả các bảng của POSTGRES_DSN
def get_postgres_tables(dsn):
    with psycopg2.connect(dsn) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public'
            """)
            tables = [row[0] for row in cur.fetchall()]
    return tables
# =====================================================
print(get_postgres_tables(POSTGRES_DSN))

['inventory_entries', 'options', 'teams', 'users', 'audit_log']


In [5]:
# Lấy tên cột của bảng table_postgresql_name trong PostgreSQL
table_postgresql_name = 'inventory_entries'

def get_postgres_table_columns(dsn, table_name):
    with psycopg2.connect(dsn) as conn:
        with conn.cursor() as cur:
            cur.execute(f"""
                SELECT column_name
                FROM information_schema.columns
                WHERE table_name = %s
            """, (table_name,))
            columns = [row[0] for row in cur.fetchall()]
    return columns
print(get_postgres_table_columns(POSTGRES_DSN, table_postgresql_name))

['id', 'component_id', 'component_name', 'group_name', 'process', 'model', 'size', 'unit', 'team_id', 'material', 'storage_location', 'invoice', 'modinvoice', 'status', 'note', 'quantity', 'movement_type', 'created_at', 'created_by', 'updated_at', 'updated_by', 'search_vector']


In [6]:
# đọc dữ liệu table_postgresql_name từ PostgreSQL
def read_postgres_table(dsn, table_name):
    with psycopg2.connect(dsn) as conn:
        with conn.cursor() as cur:
            cur.execute(f"SELECT * FROM {table_name}")
            rows = cur.fetchall()
    return rows

# print(read_postgres_table(POSTGRES_DSN, table_postgresql_name))
# chuyển dữ liệu table_postgresql_name thành DataFrame
def postgres_table_to_dataframe(dsn, table_name):
    with psycopg2.connect(dsn) as conn:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
    return df
df = postgres_table_to_dataframe(POSTGRES_DSN, table_postgresql_name)
df

C:\Users\sscn203\AppData\Local\Temp\ipykernel_23428\554903517.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id,component_id,component_name,group_name,process,model,size,unit,team_id,material,...,modinvoice,status,note,quantity,movement_type,created_at,created_by,updated_at,updated_by,search_vector


In [7]:
def read_sqlite_table(table_name_db):
    print(f"Đang đọc {table_name_db} từ SQLite...")
    if not os.path.exists(SQLITE_DB):
        raise FileNotFoundError(SQLITE_DB)
    
    conn = sqlite3.connect(SQLITE_DB)
    query = f"SELECT * FROM {table_name_db}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Đọc: {len(df)} dòng.")
    return df

In [8]:
df_user = read_sqlite_table(TABLE_USER)
df_check_id_user = df_user.copy()
df_check_id_user = df_check_id_user.rename(columns={'id': 'created_by'})
# xóa cột id
df_user = df_user.drop(columns=['id'])
df_user['password'] = hashed_password
df_user['created_at'] = pd.Timestamp.now()
df_user

Đang đọc user_inventory từ SQLite...
Đọc: 15 dòng.


,username,password,level,created_at
0,14-1733,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,5,2025-11-12 14:22:36.926555
1,13-1538,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
2,13-1692,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
3,14-1789,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
4,14-1805,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
5,15-1019,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
6,20-1007,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
7,19-1111,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
8,21-1047,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555
9,14-1825,$2b$12$hcofTEOdKpWMw.oWx4TayePQ7RzcSDfxsZQDR5L...,2,2025-11-12 14:22:36.926555


In [9]:
# đọc dữ liệu bảng input_inventory từ SQLite
df_in = read_sqlite_table(TABLE_INPUT)
df_in['movement_type'] = 'in'
df_in

Đang đọc input_inventory từ SQLite...
Đọc: 12480 dòng.


,id,component_id,component_name,process,model,size,quantity,unit,groups,material,storage_location,invoice,desinvoice,status,note,date_time,username,movement_type
0,52,072009,Dây điện,Other,Other,,1,pcs,Electric Wires,Plastic,Box 072,None,None,Available,.,2025-09-11 14:57:21,14-1733,in
1,54,072011,Dây kết nối 6 sợi,Other,Other,,20,pcs,Electric Wires,Plastic,Box 072,None,None,Available,,2025-07-09 11:13:01,12-6556,in
2,55,072012,Dây nguồn 220v đen,Other,Other,,3,pcs,Electric Wires,Plastic,Box 072,None,None,Available,\n1pcs_CBW210564-NPAC30_EQ21051244-1\n--------...,2025-04-05 17:18:38,14-1781,in
3,56,072013,Cáp Dislayport 2,Other,Other,,1,pcs,Electric Wires,Plastic,Box 072,None,None,Available,,2025-06-02 07:57:39,12-6556,in
4,58,072015,Dây kết nối 6 sợi,Other,Other,,7,pcs,Electric Wires,Plastic,Box 072,None,None,Available,,2025-09-03 09:40:38,14-1733,in
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12475,14056,VE12147,Pad - Tool Bend,Bending,Other,,1,pcs,Setup Parts,Metal,Vendor VE12,,,Available,,2025-11-11 09:03:45,14-1825,in
12476,14057,VE12148,Pad - Tool Bend,Bending,Other,,1,pcs,Setup Parts,Metal,Vendor VE12,,,Available,,2025-11-11 09:03:28,14-1825,in
12477,14058,VE12149,Pad - Tool Bend,Bending,Other,,1,pcs,Setup Parts,Metal,Vendor VE12,,,Available,1pcs_CBW191223-NJUWC40_EQ1912019-60 (lấy tool)...,2025-11-11 16:30:57,14-1805,in
12478,14059,VE12150,Pad - Tool Bend,Bending,Other,,1,pcs,Setup Parts,Metal,Vendor VE12,,,Available,,2025-11-11 09:04:06,14-1825,in


In [10]:
# đọc dữ liệu bảng output_inventory từ SQLite
df_out = read_sqlite_table(TABLE_OUTPUT)
df_out['movement_type'] = 'out'
df_out

Đang đọc output_inventory từ SQLite...


Đọc: 1802 dòng.


,id,component_id,component_name,process,model,size,quantity,unit,groups,material,storage_location,invoice,desinvoice,status,note,date_time,username,movement_type
0,26,003118,Pad nối mass cho robot LD & ULD ACFauto,ACF,Other,,20,pcs,Setup Parts,Metal,Box 003,None,None,None,Xuất cho KTTB thay thế cho 5 line ACF trả lại ...,2024-12-03 16:34:27,14-1733,out
1,30,017022,Pad pickup FPC,ACF,BE,,1,set,Setup Parts,Plastic,Box 017,None,None,None,chuyển qua invoice CBW210702 (BRX),2025-01-09 08:34:21,14-1733,out
2,31,060097,Nozzle ULD ACFauto,ACF,BRW,,8,pcs,Setup Parts,Plastic,Box 060,None,None,None,chuyển qua invoice CBW210702 (BRX),2025-01-09 08:34:59,14-1733,out
3,34,060014,shuttle,ACF,BRW,,2,set,Setup Parts,Metal,Box 060,None,None,None,chuyển qua invoice CBW210702 (BRX),2025-01-09 08:38:07,14-1733,out
4,35,060139,invoice : CBW210702,ACF,BRX,,1,set,Setup Parts,Metal,Box 060,None,None,None,Xuất linh kiện theo line ACF24 (ngày 17.01.2025),2025-01-17 15:41:38,14-1733,out
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,2027,023085,Móng kẹp ST9,AVI-AIO,Other,,1,pcs,Setup Parts,Plastic,Box 023,,,Unavailable,Xuất ngày 22/10/2025\n1pcs_CBW210539-NPAMP_EQ2...,2025-11-10 14:21:56,12-6556,out
1798,2028,062071,Móng kẹp ST1-ST13,AVI-AIO,Other,,1,pcs,Setup Parts,Plastic,Box 062,,,Available,Xuất ngày 22/10/2025\n1pcs_CBW210539-NPAMP_EQ2...,2025-11-10 14:23:01,12-6556,out
1799,2029,VE06065,Jig index,AVI-AIO,Other,,1,pcs,Setup Parts,Plastic,Vendor VE06,None,,Unavailable,Xuất ngày 22/10/2025\n1pcs_CBW210539-NPAMP_EQ2...,2025-11-10 14:23:50,12-6556,out
1800,2030,VE01029,Móng kẹp định vị ST1-ST9 PAC-K-217,AVI-AIO,Other,,3,pcs,Setup Parts,Plastic,Vendor VE01,,,Available,Xuất ngày 22/10/2025\n3pcs_CBW210539-NPAMP_EQ2...,2025-11-11 11:19:18,12-6556,out


In [11]:
# ghép 2 dataframe lại với nhau không bỏ trùng, lấy hết
df_inventory_entries = pd.concat([df_in, df_out], ignore_index=True)
# các cột cần chuyển
cols_to_convert = ['groups', 'process', 'model', 'material']

def wrap_value(x):
    x_str = str(x).strip()
    if " " in x_str:  # nếu có khoảng trắng
        return f'{{"{x_str}"}}'
    else:
        return f'{{{x_str}}}'

for col in cols_to_convert:
    df_inventory_entries[col] = df_inventory_entries[col].apply(wrap_value)

# rename cột "groups" thành "group_name"
df_inventory_entries = df_inventory_entries.rename(columns={"groups": "group_name","date_time":"created_at", "desinvoice" :"modinvoice"})

df_inventory_entries['team_id'] = TEAM_ID

# Xử lý cột username nếu ô nào empty thì cho "14-1733"
df_inventory_entries['username'] = df_inventory_entries['username'].fillna('14-1733')
# ghép 2 dataframe để lấy df_inventory_entries là chính với df_check_id_user so sánh với cột username
df_inventory_entries = pd.merge(df_inventory_entries, df_check_id_user[['created_by', 'username']], how='left')
# xử lý cột created_by nếu có giá trị NaN thì gán là 2
df_inventory_entries['created_by'] = df_inventory_entries['created_by'].fillna(2)
# format cột created_by là int
df_inventory_entries['created_by'] = df_inventory_entries['created_by'].astype(int)
# sửa lại cột created_by nếu giá trị nào lơn hơn 35 thì sưa thành 2
df_inventory_entries.loc[df_inventory_entries['created_by'] > 35, 'created_by'] = 2
# xóa cột id
df_inventory_entries = df_inventory_entries.drop(columns=["id", "username"])
df_inventory_entries

,component_id,component_name,process,model,size,quantity,unit,group_name,material,storage_location,invoice,modinvoice,status,note,created_at,movement_type,team_id,created_by
0,072009,Dây điện,{Other},{Other},,1,pcs,"{""Electric Wires""}",{Plastic},Box 072,None,None,Available,.,2025-09-11 14:57:21,in,1,1
1,072011,Dây kết nối 6 sợi,{Other},{Other},,20,pcs,"{""Electric Wires""}",{Plastic},Box 072,None,None,Available,,2025-07-09 11:13:01,in,1,15
2,072012,Dây nguồn 220v đen,{Other},{Other},,3,pcs,"{""Electric Wires""}",{Plastic},Box 072,None,None,Available,\n1pcs_CBW210564-NPAC30_EQ21051244-1\n--------...,2025-04-05 17:18:38,in,1,2
3,072013,Cáp Dislayport 2,{Other},{Other},,1,pcs,"{""Electric Wires""}",{Plastic},Box 072,None,None,Available,,2025-06-02 07:57:39,in,1,15
4,072015,Dây kết nối 6 sợi,{Other},{Other},,7,pcs,"{""Electric Wires""}",{Plastic},Box 072,None,None,Available,,2025-09-03 09:40:38,in,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14277,023085,Móng kẹp ST9,{AVI-AIO},{Other},,1,pcs,"{""Setup Parts""}",{Plastic},Box 023,,,Unavailable,Xuất ngày 22/10/2025\n1pcs_CBW210539-NPAMP_EQ2...,2025-11-10 14:21:56,out,1,15
14278,062071,Móng kẹp ST1-ST13,{AVI-AIO},{Other},,1,pcs,"{""Setup Parts""}",{Plastic},Box 062,,,Available,Xuất ngày 22/10/2025\n1pcs_CBW210539-NPAMP_EQ2...,2025-11-10 14:23:01,out,1,15
14279,VE06065,Jig index,{AVI-AIO},{Other},,1,pcs,"{""Setup Parts""}",{Plastic},Vendor VE06,None,,Unavailable,Xuất ngày 22/10/2025\n1pcs_CBW210539-NPAMP_EQ2...,2025-11-10 14:23:50,out,1,15
14280,VE01029,Móng kẹp định vị ST1-ST9 PAC-K-217,{AVI-AIO},{Other},,3,pcs,"{""Setup Parts""}",{Plastic},Vendor VE01,,,Available,Xuất ngày 22/10/2025\n3pcs_CBW210539-NPAMP_EQ2...,2025-11-11 11:19:18,out,1,15


In [12]:
# module chuyển dữ liệu SQLite users -> PostgreSQL
def insert_user_to_postgres(df):
    print("Kết nối PostgreSQL...")
    conn = psycopg2.connect(POSTGRES_DSN)
    cur = conn.cursor()

    # 2. CHÈN DỮ LIỆU MỚI - DÙNG executemany
    records = [
        (
            row['username'],
            row['email'],
            row['password'],
            row['role'],
            row['team_id'],
            row['is_active'],
            row['created_at']
        )
        for _, row in df.iterrows()
    ]
    
    insert_sql = """
        INSERT INTO users (username, email, password, role, team_id, is_active, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (username) DO UPDATE SET
            email = EXCLUDED.email,
            password = EXCLUDED.password,
            role = EXCLUDED.role,
            team_id = EXCLUDED.team_id,
            is_active = EXCLUDED.is_active,
            created_at = EXCLUDED.created_at
    """
    
    print(f"Chèn {len(records)} bản ghi bằng executemany...")
    cur.executemany(insert_sql, records)

    conn.commit()
    cur.close()
    conn.close()
    print("HOÀN TẤT!")

In [13]:
# module chuyển dữ liệu SQLite in/out -> PostgreSQL
def insert_inventory_to_postgres(df):
    print("Kết nối PostgreSQL...")
    conn = psycopg2.connect(POSTGRES_DSN)
    cur = conn.cursor()

    # 2. CHÈN DỮ LIỆU MỚI - DÙNG executemany
    records = [
        (
            row['component_id'],
            row['component_name'],
            row['group_name'],
            row['process'],
            row['model'],
            row['size'],
            row['unit'],
            row['team_id'],
            row['material'],
            row['storage_location'],
            row['invoice'],
            row['modinvoice'],
            row['status'],
            row['note'],
            row['quantity'],
            row['movement_type'],
            row['created_at'],
            row['created_by']
        )
        for _, row in df.iterrows()
    ]
    
    insert_sql = """
        INSERT INTO inventory_entries (component_id, component_name, group_name, process, model, size, unit, team_id, material, storage_location, invoice, modinvoice, status, note, quantity, movement_type, created_at, created_by)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    print(f"Chèn {len(records)} bản ghi bằng executemany...")
    cur.executemany(insert_sql, records)

    conn.commit()
    cur.close()
    conn.close()
    print("HOÀN TẤT!")

In [14]:
insert_inventory_to_postgres(df_inventory_entries)

Kết nối PostgreSQL...
Chèn 14282 bản ghi bằng executemany...
HOÀN TẤT!


In [15]:
# chuyển dữ liệu SQLite users -> PostgreSQL
# insert_user_to_postgres(df_user)